In [ ]:
# import the packages you need
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import astropy.constants as C
from scipy import integrate

This notebook calculates and plots various cosmological distance measures as a function of redshift using a flat Lambda-CDM cosmology.

We start by importing the necessary libraries: `matplotlib.pyplot` for plotting, `numpy` for numerical operations, `astropy.units` and `astropy.constants` for handling physical units and constants, and `scipy.integrate` for numerical integration.

This cell sets some parameters for the plot to make it more visually appealing.

In [ ]:
### This cell serves no purpose other than making your plot a bit pretty ###
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['font.size'] = 18

Here, we define the cosmological parameters for a flat Lambda-CDM model:
- `H0`: The Hubble constant, which describes the expansion rate of the universe today.
- `c`: The speed of light.
- `Omega_m0`: The density parameter for matter today.
- `Omega_lambda0`: The density parameter for dark energy today. For a flat universe, $\Omega_{m0} + \Omega_{\Lambda0} = 1$.

In [ ]:
H0 = 70 * u.km / u.s / u.Mpc
c = C.c  # speed of light

# Define cosmology parameters
Omega_m0 = 0.3  # Matter density today
Omega_lambda0 = 1 - Omega_m0  # Dark energy density today (for flat cosmology)

This cell defines the functions used to calculate the different cosmological distances and the age of the universe at a given redshift.

- `Ez_inverse(z, Om0, Ol0)`: This function calculates the inverse of the Hubble parameter as a function of redshift, which is a key component in the integrals for calculating distances and time in a Friedmann-Lemaître-Robertson-Walker (FLRW) universe. The equation is given by:

$$ E(z) = \sqrt{\Omega_{m0}(1+z)^3 + \Omega_{\Lambda0}} $$

- `angular_diameter_distance(z)`: This function calculates the angular diameter distance, which relates the physical size of an object to its observed angular size. The formula is:

$$ D_A(z) = \frac{c}{H_0(1+z)} \int_0^z \frac{dz'}{E(z')} $$

- `luminosity_distance(z)`: This function calculates the luminosity distance, which relates the intrinsic luminosity of an object to its observed flux. The formula is:

$$ D_L(z) = \frac{c(1+z)}{H_0} \int_0^z \frac{dz'}{E(z')} = (1+z)^2 D_A(z) $$

- `comoving_distance(z)`: This function calculates the comoving distance, which is the distance between two objects in the expanding universe that remains constant with time if the objects are not moving relative to the Hubble flow. The formula is:

$$ D_C(z) = \frac{c}{H_0} \int_0^z \frac{dz'}{E(z')} $$

- `time(z)`: This function calculates the age of the universe at a given redshift. The formula is:

$$ t(z) = \frac{1}{H_0} \int_z^\infty \frac{dz'}{(1+z')E(z')} $$

- `proper_distance_traveled(z)`: This function calculates the proper distance traveled by light from redshift $z$ to $z=0$. This is also known as the light-travel distance or lookback distance. The formula is:

$$ D_{LT}(z) = c \int_0^z \frac{dz'}{(1+z')E(z')} = c H_0 t(z) $$

In [ ]:
# Function to integrate for comoving distance and time
def Ez_inverse(z, Om0, Ol0):
    return 1 / np.sqrt(Om0 * (1 + z)**3 + Ol0)

@np.vectorize
def angular_diameter_distance(z):
    # Integral for comoving distance
    integral_result = integrate.quad(Ez_inverse, 0, np.max(z), args=(Omega_m0, Omega_lambda0))[0]
    dc = (c / H0 * integral_result)
    return dc.to(u.kpc).value / (1 + z)

@np.vectorize
def luminosity_distance(z):
    # Integral for comoving distance
    integral_result = integrate.quad(Ez_inverse, 0, np.max(z), args=(Omega_m0, Omega_lambda0))[0]
    dl = (c / H0 * integral_result)
    return dl.to(u.kpc).value * (1 + z)

@np.vectorize
def comoving_distance(z):
    # Integral for comoving distance
    integral_result = integrate.quad(Ez_inverse, 0, np.max(z), args=(Omega_m0, Omega_lambda0))[0]
    dc = (c / H0 * integral_result)
    return dc.to(u.kpc).value

@np.vectorize
def time(z):
    # Integral for cosmic time
    integral_result = integrate.quad(lambda x, Om0, Ol0: 1 / ((1 + x) * np.sqrt(Om0 * (1 + x)**3 + Ol0)), z, np.inf, args=(Omega_m0, Omega_lambda0))[0]
    t = (1 / H0 * integral_result)
    return t.to(u.s)

@np.vectorize
def proper_distance_traveled(z):
    # Light-travel (lookback) distance: c ∫_0^z dz' / [(1+z') H(z')]
    integrand = lambda x, Om0, Ol0: 1 / ((1 + x) * np.sqrt(Om0 * (1 + x)**3 + Ol0))
    integral_result = integrate.quad(integrand, 0, z, args=(Omega_m0, Omega_lambda0))[0]
    dlt = (c / H0) * integral_result
    return dlt.to(u.kpc).value


In [ ]:
# produce an array of redshift from 0 to 2 in increments of 0.01
# pick a range for redshift that makes the most sense to you
min_redshift = 0 # pick the min redshift
max_redshift = 2 # pick the max redshift

z = np.arange(min_redshift, max_redshift, 0.01)
fig, ax = plt.subplots(figsize=(7, 5))

ax.loglog(z, angular_diameter_distance(z), label=r"$D_A$ [kpc]")
ax.loglog(z, comoving_distance(z), label=r"r(z)[kpc]")
ax.loglog(z, luminosity_distance(z), label=r"$D_L$ [kpc]")
ax.loglog(z, proper_distance_traveled(z), label=r"$D_s$ [kpc]")
ax.set_xlabel("Redshift")
ax.set_ylabel("Distance (kpc)")
ax.set_title(f"$\\Omega_m = {Omega_m0}, \\Omega_\\Lambda = {Omega_lambda0}$")
plt.legend()

In [ ]:
# produce an array of redshift from 0 to 1300 in increments of 1
# pick a range for redshift that makes the most sense to you
min_redshift = 0 # pick the min redshift
max_redshift = 1300 # pick the max redshift

z = np.arange(min_redshift, max_redshift,0.1)
fig, ax = plt.subplots(figsize=(7, 5))

ax.loglog(z, angular_diameter_distance(z), label=r"$D_A$ [kpc]")
ax.loglog(z, comoving_distance(z), label=r"r(z)[kpc]")
ax.loglog(z, luminosity_distance(z), label=r"$D_L$ [kpc]")
ax.loglog(z, proper_distance_traveled(z), label=r"$D_s$ [kpc]")
ax.set_xlabel("Redshift")
ax.set_ylabel("Distance (kpc)")
ax.set_title(f"$\\Omega_m = {Omega_m0}, \\Omega_\\Lambda = {Omega_lambda0}$")
plt.legend()

## Explore and Reflect

Now that you have the code to calculate and plot cosmological distances, try the following:

1. **Vary $\Omega_{m0}$**: Change the value of `Omega_m0` in the cell where the cosmological parameters are defined. Remember that for a flat universe, $\Omega_{\Lambda0} = 1 - \Omega_{m0}$. Rerun the subsequent cells to see how the different distance measures change with a different matter density.  
2. **Create Plots**: Generate plots similar to the ones above for a few different values of $\Omega_{m0}$ (e.g., 0.1, 0.2, 0.5, 0.9, etc).  Be sure to create new cells for each plot, so that your submitted notebook shows your experiments.
3. **Write a Reflection**: In a new text cell, write a short reflection on what you observe. How do the different distance measures change as you vary $\Omega_{m0}$? What does this tell you about the impact of matter density on the expansion of the universe and the relationship between redshift and distance?

In [ ]:
### INSERT YOUR CODE HERE!

**Write your reflection here!**